In [ ]:
import coursierapi._
interp.repositories() ++= Seq(MavenRepository.of("https://jitpack.io"))

In [ ]:
import $ivy.`com.github.dzufferey::scadla:0.1.1`
import scadla._
import InlineOps._
import EverythingIsIn.{millimeters, degrees}
import scadla.utils.thread.ISO
import scadla.utils.CenteredCube
import squants.space.{Length, Millimeters}
import scala.language.postfixOps
import squants.space.LengthConversions._
import scadla.backends.OpenSCAD // for rendering (getting a mesh)
import scadla.backends.almond.Viewer // to show the mesh in jupyter/almond

In [ ]:
import $ivy.`com.github.dzufferey::libgcode:0.1-SNAPSHOT`
import libgcode.utils.{Viewer => NcViewer}
import libgcode._
import libgcode.extractor._
import libgcode.generator._
import libgcode.utils._
import libgcode.utils.geometry2D._
import scala.collection.mutable.ArrayBuffer

# New Z-axis for my cnc

## Dimensions for mechanical components

In [ ]:
// linear rails (MGN 15C)
object LinearRail {
    val totalHeight = 16 mm // surface to surface distance includes rail + guide
    val width = 15 mm
    val height = 10 mm
    val holesDistance = 40 mm
    val holesRadius = ISO.M3
    def screwPositions(length: Length) = { // x coord of the mounting screw
        val n = ((length - holesRadius * 2) / holesDistance).floor.toInt
        val offset = (length - holesDistance * n) / 2
        (0 until n).map( i => offset + holesDistance * i)
    }
    // extend in x direction [0, length], y centered at 0, back against xy plane
    def rail(length: Length) = {
        val r = Cube(length, width, height).moveY(- width / 2)
        val screw = Cylinder(holesRadius, height) + Cylinder(holesRadius * 2, holesRadius).moveZ(height - holesRadius + 0.1)
        val hs = screwPositions(length).map( x => screw.moveX(x) )
        r -- hs
    }
    def holePattern(length: Length, h: Length = 10) = {
        val screw = Cylinder(holesRadius, h+1).moveZ(-h)
        screwPositions(length).map( x => screw.moveX(x) )
    }
}
object LinearGuide {
    val length = 42.1 mm
    val width = 32 mm
    val height = 12 mm
    val holesLength = 20 mm
    val holesWidth = 25 mm
    val holesRadius = ISO.M3
    // centered at (0,0), back at 0 and guide facing up
    val holePattern = Seq(( holesLength / 2,  holesWidth / 2),
                          (-holesLength / 2,  holesWidth / 2),
                          ( holesLength / 2, -holesWidth / 2),
                          (-holesLength / 2, -holesWidth / 2))
    val model = {
        val c0 = CenteredCube.xy(length, width, height)
        val c1 = c0 - CenteredCube.xy(length, LinearRail.width, LinearRail.height).moveZ(4)
        c1 -- holePattern.map{ case (x,y) => Cylinder(holesRadius, height).move(x, y, 0) } // make througholes to see align
    }
}
// ballscrew
object BallScrew {
    // https://www.dold-mechatronik.de/Spindelmuttergehaeuse-DSG1605-fuer-Kugelumlaufspindel-SFU1605-SFU1610
    val nutHeight = 40 mm
    val nutWidth = 52 mm
    val nutLength = 40 mm
    val nutHoles = ISO.M5
    val nutHolesDistanceWidth = 40 mm
    val nutHolesDistanceLength = 24 mm
    val nutHolesPattern = Seq(( nutHolesDistanceLength / 2,  nutHolesDistanceWidth / 2),
                              (-nutHolesDistanceLength / 2,  nutHolesDistanceWidth / 2),
                              ( nutHolesDistanceLength / 2, -nutHolesDistanceWidth / 2),
                              (-nutHolesDistanceLength / 2, -nutHolesDistanceWidth / 2))
    // https://www.dold-mechatronik.de/Festlagereinheit-FLB20-3200-Easy-Mechatronics-System-1620B
    val endHeight = 51 mm
    val endWidth = 70 mm
    val endDepth = 20 mm
    val endHolesDistance = 55 mm
    val endHoles = ISO.M6
    val endHolesPattern = Seq(-endHolesDistance / 2, endHolesDistance / 2)
    val bottomToRotationAxis = 31 mm
    val bearingUnit = {
        val c0 = CenteredCube.xy(endDepth, 40, endHeight)
        val c1 = c0 + CenteredCube.xy(endDepth, endWidth, 10)
        val c2 = c1 -- endHolesPattern.map( y => Cylinder(endHoles, 11).moveY(y) )
        c2
    }
    // length is the distance between mounting screw
    def model(l: Length) = {
        bearingUnit + bearingUnit.moveX(l) + Cylinder(10, l).rotateY(90).moveZ(bottomToRotationAxis)
    }
}
// spindle clamp: https://www.dold-mechatronik.de/Eurohalsaufnahme-65x75x20
object SpindleClamp {
    val width = 65 mm
    val height = 20 mm
    val depth = 75 mm
    val backToCenter = 42 mm
    val centerDiameter = 43 mm
    val centerGap = 2 mm
    val mountingHoles = ISO.M8
    val holesPosition = Seq(7.5 mm, 32.5 mm, 57.5 mm)
    val holesHeight = 10 mm
    val model = {
        val x0 = Cube(width, depth, height)
        val x1 = x0 - Cylinder(centerDiameter/2, height).move(width/2, backToCenter, 0)
        val x2 = x1 - CenteredCube.x(centerGap, depth, height).move(width/2, backToCenter + 1, 0)
        val screw = Cylinder(mountingHoles, 17).rotateX(-90)
        val x3 = x2 -- holesPosition.map( x => screw.move(x, 0, height / 2) )
        x3
    }
}

def holesPosition0(n: Int, start: Length, end: Length) = {
    val delta = (end - start) / (n-1)
    (0 until n).map( i => start + delta * i )
}

def holesPosition1(n: Int, length: Length, distanceToBorder: Length) = {
    holesPosition0(n, distanceToBorder, length - distanceToBorder)
}

## XY table (already made)

In [ ]:
object Table {
    val mountingHolesWidth = 160 mm
    val mountingHolesDepth = 80 mm
    val mountingHoles = ISO.M8
    val xTravel = 300 mm
    val yTravel = 200 mm
    val distBetweenPlates = BallScrew.nutHeight / 2 +  BallScrew.bottomToRotationAxis
    val plateThickness = 20 mm
    val totalHeigth = plateThickness * 3 + distBetweenPlates * 2
    def placeholder(x: Double = 0.5, y: Double = 0.5) = {
        val lower = CenteredCube.xy(550, 200, plateThickness)
        val mz = plateThickness + distBetweenPlates
        val xPos = xTravel * (0.5 - x)
        val middle = CenteredCube.x(200, 350, plateThickness).move(xPos, -130, mz)
        val uz = mz * 2
        val yPos = yTravel * (1 - y)
        val upper = CenteredCube.xy(300, 200, plateThickness).move(xPos, yPos - 55, uz)
        lower + middle + upper
    }
    val xOffset = 0 mm
    val yOffset = 45 mm
}

In [ ]:
val mesh = OpenSCAD(Table.placeholder())
Viewer(mesh)

In [ ]:
case class HoleSpec(
    x: Length,
    y: Length,
    radius: Length,
    depth: Length,
    counterBore: Boolean = false,
    threaded: Boolean = false
) {
    protected def counterBoreRadius = if (counterBore) 2*radius.to(Millimeters) else 0
    protected def counterBoreDepth = if (counterBore) radius.to(Millimeters) else 0
    protected def _x = x.to(Millimeters)
    protected def _y = y.to(Millimeters)
    protected def _radius = radius.to(Millimeters)
    protected def _depth = depth.to(Millimeters)
    def model(z: Length, extra: Length = 0.1 mm) = {
        val m0 = Cylinder(radius, depth + extra)
        val m1 = Cylinder(counterBoreRadius, counterBoreDepth + extra).moveZ(depth - counterBoreDepth)
        val m2 = m0 + m1
        m2.move(x, y, z - depth)
    }
    def spot(z: Length, ignoreCounterBore: Boolean = false)(implicit conf: Config): Seq[Command] = {
        val _z = z.to(Millimeters)
        if (ignoreCounterBore) {
            Drill(_x, _y, _z, conf.endmillRadius)
        } else {
            Drill(_x, _y, _z - counterBoreDepth, conf.endmillRadius)
        }
    }
    def drill(z: Length, ignoreCounterBore: Boolean = false)(implicit conf: Config): Seq[Command] = {
        val _z = z.to(Millimeters)
        if (ignoreCounterBore) {
            Drill(_x, _y, _z, _depth)
        } else {
            Drill(_x, _y, _z - counterBoreDepth, _depth - counterBoreDepth)
        }
    }
    protected def threadingHole(radius: Length) = {
        implicit val tolerance = (1e-6 mm)
        if (radius ~= ISO.M1_6) 1.25 / 2
        else if (radius ~= ISO.M2) 0.8
        else if (radius ~= ISO.M2_5) 1
        else if (radius ~= ISO.M3) 1.25
        else if (radius ~= ISO.M4) 1.6
        else if (radius ~= ISO.M5) 2.1
        else if (radius ~= ISO.M6) 2.5
        else if (radius ~= ISO.M8) 3.4
        else if (radius ~= ISO.M10) 4.25
        else if (radius ~= ISO.M12) 5
        else radius.to(Millimeters) * 0.4
    }
    def toolpath(z: Length, ignoreShaft: Boolean = false)(implicit conf: Config): Seq[Command] = {
        val _z = z.to(Millimeters)
        val buffer = scala.collection.mutable.ArrayBuffer.empty[Command]
        if (counterBore) {
            buffer ++= Hole(_x, _y, _z, counterBoreRadius, counterBoreDepth)
        }
        if (!ignoreShaft) {
            var r = if (threaded) threadingHole(radius) else _radius
            buffer ++= Hole(_x, _y, _z - counterBoreDepth, r, _depth - counterBoreDepth)
        }
        buffer.toSeq
    }
}

In [ ]:
abstract class Part {
    def simple: Boolean = false
    def model: Solid
    def info: String
    def contour(width: Length, onionSkin: Length = (0.5 mm))(implicit conf: Config): Seq[Command]
    def chamfer(implicit conf: Config): Seq[Command]
    def spotHoles(ignoreCounterBore: Boolean = false)(implicit conf: Config): Seq[Command]
    def drillHoles(ignoreCounterBore: Boolean = false)(implicit conf: Config): Seq[Command]
    def boreHoles(ignoreShaft: Boolean = false)(implicit conf: Config): Seq[Command]
}

// a simple part is milled from one part with just a rectangle with some holes
abstract class SimplePart(name: String) extends Part {
    override def simple = true
    
    def width: Length /* X */
    def depth: Length /* Y */
    def height: Length /* Z */
    def holes: Seq[HoleSpec]
    
    def model: Solid = {
        Cube(width, depth, height) -- holes.map( _.model(height) )
    }
    
    def corners(r: Double) = {
        Seq[(Double, Double, Double)] = Seq(
            (0, 0, r),
            (width.to(Millimeters), 0, r),
            (width.to(Millimeters), depth.to(Millimeters), r),
            (0, depth.to(Millimeters), r)
        )
    }
    
    def contour(width: Length, onionSkin: Length)(implicit conf: Config): Seq[Command] = {
        ???
    }
    
    def chamfer(implicit conf: Config): Seq[Command] = {
        val r = conf.endmillRadius
        Contour(r), height.to(Millimeters) - r)
    }
    
    def spotHoles(ignoreCounterBore: Boolean)(implicit conf: Config): Seq[Command] = {
        holes.flatMap( _.spot(height, ignoreCounterBore) )
    }
    
    def drillHoles(ignoreCounterBore: Boolean)(implicit conf: Config): Seq[Command] = {
        holes.flatMap( _.drill(height, ignoreCounterBore) )
    }
    
    def boreHoles(ignoreShaft: Boolean)(implicit conf: Config): Seq[Command] = {
        holes.flatMap( _.toolpath(height, ignoreShaft) )
    }
    
    def extraInfo: String = ""
    
    def info = {
        s"""|$name:
            |  width = $width
            |  depth = $depth
            |  height = $height
            |${extraInfo}""".stripMargin
    }
}

In [ ]:
//TODO more comlex parts
//specifying the base shape ....

## Draft

In [ ]:
val carriageHeight = 150 mm
val pillarInnerspace = BallScrew.nutWidth.max(BallScrew.endWidth) + 10 //TODO smaller endwidth
val expectedTravel = 150 mm

//TODO where to put the z-axis stop

object Pillar {
    val width = 15 mm
    val depth = 60 mm
    val height = Table.totalHeigth +
                 15 /* slots for t-nuts */ +
                 30 + /* spindle collet to collar */
                 carriageHeight + /* carriage heigth */
                 expectedTravel /* expected z travel */
    val railLength = height
    
    val kneeDepth = 50 mm
    val kneeHeight = 20 mm
    
    def totalDepth = depth + kneeDepth
    
    val baseHoles = ISO.M8
    val baseHolesNumber = 3
    val baseHolesPositions = {
        val l = depth + kneeDepth
        val toBorder = baseHoles * 2.5
        val dist = (l - toBorder * 2) / (baseHolesNumber - 1)
        Seq(toBorder, toBorder + dist + 5, toBorder + dist * 2)
    }
    
    val frontDepth = 6 mm
    val frontHole = ISO.M3
    val frontHeight = height - kneeHeight
    val frontHolesPositions = holesPosition1(6, frontHeight, frontHole * 3 + 10).map( _ + kneeHeight )
    
    //TODO should fillet the knee
    //TODO rail length should be somewhat standard
    
    def model = {
        val p = Cube(width, depth, height)
        val k = Cube(width, kneeDepth, kneeHeight)
        val s0 = p + k.moveY(-kneeDepth)
        // holes to attach
        // (1) the base
        val s1 = s0 -- baseHolesPositions.map( y => Cylinder(baseHoles, 15).move(width/2, y - kneeDepth, 0) )
        // (2) the front
        val s2 = s1 -- frontHolesPositions.map( z => Cylinder(frontHole, width).rotateY(90).move(0, frontDepth / 2 , z) )
        // (3) the rails
        val s3 = s2 -- LinearRail.holePattern(railLength,width).map( _.rotateY(-90).moveY(depth/2) )
        s3
    }
    
    def info = {
        s"""|Pillar (2x):
            |  width = $width
            |  depth = $depth
            |  height = $height
            |  rail length =  $railLength""".stripMargin
    }
}

In [ ]:
val mesh = OpenSCAD(Pillar.model)
Viewer(mesh)

In [ ]:
object FrontPlate {
    val width = pillarInnerspace
    val depth = Pillar.frontDepth
    val height = Pillar.height
    val ballScrewLength = expectedTravel + 200 // when I make a lower table, I'll have more travel
    
    // TODO spacer for the front plate to hold the ballscrew at the proper distance
    // TODO to mount the ballscrew attach from the back, tighter holes
    // TODO better travel / height ratio ?
    
    def model = {
        val f0 = Cube(width, depth, height)
        // holes for pillar
        val pd2 = depth / 2
        val s = Cylinder(Pillar.frontHole, 10).rotateY(90)
        val f1 = f0 -- Pillar.frontHolesPositions.map( z => s.move(0, pd2, z) + s.move(width-10, pd2, z))
        // holes for ballscrew
        val s2 = Cylinder(BallScrew.endHoles, 10).rotateX(-90)
        val h2 = height - 10
        val h3 = h2 - ballScrewLength + 10
        val f2 = f1 -- BallScrew.endHolesPattern.map( x => s2.move(width/2+x, 0, h2) + s2.move(width/2+x, 0, h3) )
        f2
    }
    
    def info = {
        s"""|FrontPlate (1x):
            |  width = $width
            |  depth = $depth
            |  height = $height
            |  ballScrewLength = $ballScrewLength""".stripMargin
    }
}

In [ ]:
val mesh = OpenSCAD(FrontPlate.model)
Viewer(mesh)

In [ ]:
//the model is upside-down
object Base extends SimplePart("Base") {
    def width = 200 mm
    def height = 20 mm
    def depth = (200 mm) /* table bottom plate */ +
                (115 mm) /* table middle plate overhang*/ +
                (20 mm) /* clearance from the top plate */ +
                Pillar.depth
    // mounting holes for the table (should be threaded!)
    val tableMountingHoles = Seq((width/2 - Table.mountingHolesWidth /2, 100 - Table.mountingHolesDepth / 2),
                                 (width/2 + Table.mountingHolesWidth /2, 100 - Table.mountingHolesDepth / 2),
                                 (width/2 - Table.mountingHolesWidth /2, 100 + Table.mountingHolesDepth / 2),
                                 (width/2 + Table.mountingHolesWidth /2, 100 + Table.mountingHolesDepth / 2))
    // countersunk holes to mount the pillar
    val pillarMountingHoles = {
        val widthDifference = (pillarInnerspace + Pillar.width) / 2
        Pillar.baseHolesPositions.map( y => (width/2 + widthDifference, depth - Pillar.totalDepth + y) ) ++
        Pillar.baseHolesPositions.map( y => (width/2 - widthDifference, depth - Pillar.totalDepth + y) )
    }
    
    def holes = {
        tableMountingHoles.map{ case (x,y) => HoleSpec(x, y, Table.mountingHoles, height, false, true) } ++
        pillarMountingHoles.map{ case (x,y) => HoleSpec(x, y, Pillar.baseHoles, height, true, false) }
    }

}

In [ ]:
val mesh = OpenSCAD(Base.model)
Viewer(mesh)

In [ ]:
object CarriageDimenions {
    val sideThickness = 10 mm
    val frontThickness = 10 mm
    val backThickness = 6 mm
    
    val frontBackClearance = 5 mm
    
    val height = carriageHeight
    val frontHeight = height
    val backHeight = BallScrew.nutLength
    val backHeightOffset = 50 mm
    val innerWidth = pillarInnerspace + Pillar.width * 2 + LinearRail.totalHeight * 2
    val outerWidth = innerWidth + sideThickness * 2
    val sideDepth = Pillar.depth +
                    frontBackClearance * 2
    val depth = sideDepth +
                frontThickness +
                backThickness
    val upperGuideHeight = LinearGuide.length / 2 + 5
    val lowerGuideHeight = height - upperGuideHeight
    val guideDepth = sideDepth / 2
    
    val frontBackHole = ISO.M4
    val frontHolesPositions = holesPosition1(4, frontHeight, frontBackHole * 3)
    val backHolesPositions = holesPosition1(3, backHeight, frontBackHole * 3)
    
    val cantileverWidth = SpindleClamp.width
    val cantileverDepth = Base.depth -
                          100 /* to middle of table */ -
                          Pillar.depth / 2 /* to linear rail */ -
                          depth / 2 /* front of carriage */ -
                          sideThickness /* frontplate */ -
                          SpindleClamp.backToCenter -
                          Table.yOffset
    val cantileverHeight = cantileverWidth // + backThickness
    val cantileverHoles = ISO.M5 // or M6 ?
    
    val cantileverHolesPositions = {
        val x = (cantileverWidth - sideThickness) / 2
        val z1 = sideThickness / 2 + 2
        val z3 = cantileverHeight - sideThickness / 2 - 2
        val z2 = (z1 + z3) / 2
        Seq((x,z1),(x,z2),(x,z3),(-x,z1),(-x,z2),(-x,z3))
    }
}

object CarriageBack extends SimplePart("Carriage Back") {
    import CarriageDimenions._
    
    def width = outerWidth
    def depth = backHeight
    def height = backThickness
    
    def holes = {
        // holes to attach to the ballscrew nut
        BallScrew.nutHolesPattern.map{ case (y,x) => HoleSpec(width / 2 + x, depth / 2 + y, BallScrew.nutHoles, height, false, true) } ++
        // holes to attach to side
        backHolesPositions.map( y => HoleSpec(sideThickness/2, y, frontBackHole, height) ) ++
        backHolesPositions.map( y => HoleSpec(width - sideThickness/2, y, frontBackHole, height) )
    }
}

object SpindlePlate extends SimplePart("Spindle Plate") {
    import CarriageDimenions._
    
    def width = cantileverWidth
    def depth = cantileverHeight + SpindleClamp.height
    def height = sideThickness
    
    def holes = {
        cantileverHolesPositions.map{ case (x,y) => HoleSpec(cantileverWidth/2+x, SpindleClamp.height + y, cantileverHoles, height) } ++
        SpindleClamp.holesPosition.map( x => HoleSpec(x, SpindleClamp.height/2, SpindleClamp.mountingHoles, height))
    }
}

object Carriage {
    import CarriageDimenions._

    val frontBackScrew = Cylinder(frontBackHole, 40).moveZ(-20).rotateX(90)
    
    def side = {
        val s0 = Cube(sideThickness, sideDepth, height)
        val guideScrew = Cylinder(LinearGuide.holesRadius, height+2).rotateY(90)
        // holes to mount the guides
        val s1 = s0 -- LinearGuide.holePattern.map{ case (x,y) => guideScrew.move(-1, guideDepth + y, upperGuideHeight + x) }
        val s2 = s1 -- LinearGuide.holePattern.map{ case (x,y) => guideScrew.move(-1, guideDepth + y, lowerGuideHeight + x) }
        // holes to mount the front
        val s3 = s2 -- frontHolesPositions.map( z => frontBackScrew.move(sideThickness/2, 0, z))
        // holes to mount the back
        val s4 = s3 -- backHolesPositions.map( z => frontBackScrew.move(sideThickness/2, sideDepth, height - backHeight - backHeightOffset + z))
        s4
    }
        
    def back = CarriageBack.model.rotateX(90).moveY(CarriageBack.height)
    
    def cantilever = {
        // 3 plates: 2 sides and top
        val side = Cube(sideThickness, cantileverDepth, cantileverHeight)
        val top = {
            val x = cantileverWidth
            val y = cantileverDepth + frontThickness * 2 //top goes over the plates where it is attached
            val z = backThickness - 2
            val c0 = Cube(x, y, z)
            val c1 = Cube(x - sideThickness*2, y - frontThickness*2, 2)
            c0 + c1.move(sideThickness, frontThickness, -2)
        }
        val shape = side.moveX(-cantileverWidth/2) +
                    side.moveX(cantileverWidth/2 - sideThickness) +
                    top.move(-cantileverWidth/2, -frontThickness, cantileverHeight)
        // holes on the back + front of the sides
        val s0 = Cylinder(cantileverHoles, 30).moveZ(-15).rotateX(90)
        val c0 = shape -- cantileverHolesPositions.map{ case(x,z) => s0.move(x,0,z) + s0.move(x,cantileverDepth,z) }
        // holes to attach the top
        val s1 = Cylinder(ISO.M3, 10).moveZ(cantileverHeight + frontThickness - 10)
        val x = (cantileverWidth  - sideThickness) / 2
        val y0 = 20 mm // TODO as param
        val y1 = cantileverDepth - 20
        val pos = Seq((x,y0),(x,y1),(-x,y0),(-x,y1))
        val c1 = c0 -- pos.map{ case (x,y) => s1.move(x,y,0) }
        c1
    }
    
    def front = {
        val f0 = Cube(outerWidth, frontThickness, frontHeight)
        // remove material to go over the pillar's knee
        val knee = pillarInnerspace + Pillar.width * 2 + 10
        val f1 = f0 - Cube(knee, frontThickness, Pillar.kneeHeight + 5).moveX((outerWidth - knee) / 2)
        // holes to attach to side
        val f2 = f1 -- frontHolesPositions.map( z => frontBackScrew.move(sideThickness/2, 0, z) +
                                                     frontBackScrew.move(outerWidth - sideThickness/2, 0, z))
        // holes to attach the cantilever
        val s = Cylinder(cantileverHoles, frontThickness).rotateX(-90)
        val f3 = f2 -- cantileverHolesPositions.map{ case (x,z) => s.move(outerWidth/2+x, 0, frontHeight - cantileverHeight + z) }
        f3
    }
    
    def spindlePlate = SpindlePlate.model.rotateX(90).moveY(SpindlePlate.height)
    
    def model = {
        val sideAndGuides = {
            val g = LinearGuide.model.rotateY(-90).rotateZ(180).moveX(sideThickness)
            val s = side + g.move(0, guideDepth, lowerGuideHeight) + g.move(0, guideDepth, upperGuideHeight)
            s.move(-outerWidth/2, 0, 0)
        }
        val spindle = spindlePlate + SpindleClamp.model.rotateZ(180).moveX(SpindleClamp.width)
        sideAndGuides +
        sideAndGuides.mirror(1,0,0) +
        front.move(-outerWidth/2, -frontThickness, 0) +
        back.move(-outerWidth/2, sideDepth, height - backHeight - backHeightOffset) +
        cantilever.move(0, -cantileverDepth -frontThickness, height - cantileverHeight) +
        spindle.move(-cantileverWidth/2, -cantileverDepth - frontThickness - sideThickness, height - (cantileverHeight + SpindleClamp.height))
    }
    
    def info = {
        s"""|Carriage (assembly):
            |  side (2x)
            |    width = $sideThickness
            |    depth = $sideDepth
            |    height = $height
            |  front (1x)
            |    width = $outerWidth
            |    depth = $frontThickness
            |    height = $frontHeight
            |  back (1x)
            |    width = $outerWidth
            |    depth = $backThickness
            |    height = $backHeight
            |  cantilever (assembly)
            |    side (2x)
            |      width = $sideThickness
            |      depth = $cantileverDepth
            |      height = $cantileverHeight
            |    top (1x)
            |      width = $cantileverWidth
            |      depth = ${cantileverDepth + frontThickness * 2}
            |      height = $backThickness
            |  spindle plate (1x)
            |    width = $cantileverWidth
            |    depth = $sideThickness
            |    height = ${cantileverHeight + SpindleClamp.height}""".stripMargin
    }
}
//TODO splindle plate should be thicker and have countersunk screw ? or go for some kind of angle brackets / side bar ?
//TODO countersunk on the front plate
//TODO save some material on the front plate

In [ ]:
val mesh = OpenSCAD(Carriage.cantilever)
Viewer(mesh)

### Together

In [ ]:
val base = Base.model.rotateY(180).move(Base.width, 0, Base.height)
val column = {
    val pr = Pillar.model + LinearRail.rail(Pillar.railLength).rotateY(-90).moveY(Pillar.depth/2)
    val p = pr.move(-pillarInnerspace / 2 - Pillar.width, 0, 0)
    p + p.mirror(1,0,0) +
    FrontPlate.model.move(-FrontPlate.width/2, 0, 0) +
    BallScrew.model(FrontPlate.ballScrewLength).rotateY(-90).rotateZ(-90).moveZ(FrontPlate.height - 10 - FrontPlate.ballScrewLength)
}
val carriage = Carriage.model
val table = Table.placeholder()
val cnc = {
    base.move(-Base.width / 2, 0 , -Base.height) +
    column.move(0, Base.depth - Pillar.depth, 0) +
    carriage.move(0, Base.depth - Carriage.sideDepth + Carriage.frontBackClearance, 200) +
    table.move(0,100,0)
}

In [ ]:
val mesh = OpenSCAD(cnc)
Viewer(mesh)

In [ ]:
OpenSCAD.toSTL(cnc, "cnc.stl")

### List of parts

In [ ]:
println(Base.info)
println(Pillar.info)
println(FrontPlate.info)
println(Carriage.info)

## Gearbox to hold the motor and handwheel

For the gear, we will use some files from Thingiverse.
The gears are CC-BY-SA by GeneralRulofDumb, see https://www.thingiverse.com/thing:10955.

The gears (original scale) are offset `12.1 mm` from the intersection of the two rotation axes.

In [ ]:
// get the bevel gear
import java.nio.file.{Paths, Files}
import java.net.{HttpURLConnection,URL}

val gearLeft = FromFile("gear_hypoid_left.stl")
val gearRight = FromFile("gear_hypoid_right.stl")

val gearLeftURL = "https://cdn.thingiverse.com/assets/08/c0/2b/0d/90/gear_hypoid_left.stl"
val gearRightURL = "https://cdn.thingiverse.com/assets/e8/ab/bd/72/48/gear_hypoid_right.stl"

In [ ]:
def getFile(address: String, file: String) = {
    if (!Files.exists(Paths.get(file))) {
        val url = new URL(address)
        val connection = url.openConnection().asInstanceOf[HttpURLConnection]
        connection.connect()
        try {
            val code = connection.getResponseCode
            if (code != 200) {
                println("error " + code)
            } else {
                url #> new File(file) !!
            }
        } finally {
            connection.close()
        }
    }
}

getFile(gearLeftURL, gearLeft)
getFile(gearRightURL, gearRight)

This part is designed to be 3D printed rather than milled.

The faceplate for a nema motor is
- `56.4 mm` wide
- holes are `47.14 mm` apart
- center flange is `38.1mm` in diameter